In [44]:
import trimet_plan_game_helper_scripts as helpers
import geopandas as gpd
import boto3
from shapely.geometry import LineString
import pandas as pd
import seaborn as sns

In [45]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"

In [46]:
gdf_points, itineraries_reduced, itinerary_routes_reduced, tries = helpers.generate_random_points_make_itinerary(tm_boundary, trimet_crs)

In [47]:
itineraries_reduced.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
0,0,5974,1254,3982,738,1443.641433,0,WALK,WALK,,Origin,600,NW Bridge Ave & Springville,"LINESTRING (-122.77528 45.58186, -122.77535 45...",16 to 90 to 72
1,0,5974,1254,3982,738,1443.641433,1,16,BUS,600,NW Bridge Ave & Springville,9888,600 Block NW Naito Parkway,"LINESTRING (-122.77152 45.58528, -122.77149 45...",16 to 90 to 72
2,0,5974,1254,3982,738,1443.641433,2,WALK,WALK,9888,600 Block NW Naito Parkway,8339,Old Town/Chinatown MAX Station,"LINESTRING (-122.67258 45.52747, -122.67255 45...",16 to 90 to 72


In [48]:
itinerary_routes_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
1,0,5974,1254,3982,738,1443.641433,1,16,BUS,600,NW Bridge Ave & Springville,9888,600 Block NW Naito Parkway,"LINESTRING (-122.77152 45.58528, -122.77149 45...",16 to 90 to 72
3,0,5974,1254,3982,738,1443.641433,3,90,TRAM,8339,Old Town/Chinatown MAX Station,8346,NE 82nd Ave MAX Station,"LINESTRING (-122.67140 45.52518, -122.67140 45...",16 to 90 to 72
5,0,5974,1254,3982,738,1443.641433,5,72,BUS,8000,NE 82nd & 82nd Ave MAX Stn/I-84,8028,SE 82nd & Ramona,"LINESTRING (-122.57910 45.53349, -122.57910 45...",16 to 90 to 72
10,1,6623,1226,4387,1010,1447.162343,3,9,BUS,7636,SW 5th & Harvey Milk,4627,SE Powell & 26th,"LINESTRING (-122.67640 45.52092, -122.67643 45...",16 to 9 to 10
12,1,6623,1226,4387,1010,1447.162343,5,10,BUS,7228,SE 26th & Powell,2561,SE Harold & 79th,"LINESTRING (-122.63963 45.49827, -122.63963 45...",16 to 9 to 10
16,2,6634,898,5107,629,968.955433,2,44,BUS,3474,N Lombard & Charleston,5996,N Vancouver & Killingsworth,"LINESTRING (-122.75177 45.58921, -122.75176 45...",16 to 44 to 72


In [50]:
itinerary_routes_reduced['route_id'] = itinerary_routes_reduced['route_id'].astype(int)

In [51]:
gtfs_routes = pd.read_csv("gtfs/routes.txt")
gtfs_routes.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order
0,1,TRIMET,1,Vermont,3,https://trimet.org/schedules/r001.htm,NaN,NaN,400
1,2,TRIMET,FX2,Division,3,https://trimet.org/schedules/r002.htm,61A60E,FFFFFF,500


In [52]:
itinerary_routes_long_name = itinerary_routes_reduced.merge(gtfs_routes[['route_id','route_long_name']], how='inner', on='route_id')

In [53]:
# Get Unique continents
color_labels = itinerary_routes_long_name['route_id'].unique()
# List of colors in the color palettes
hex_values = sns.color_palette("Set2", len(color_labels)).as_hex()
# Map continents to the colors
color_map = dict(zip(color_labels, hex_values))
color_map['WALK'] = '#bf5f58'

In [54]:
itinerary_routes_long_name['route_color'] = itinerary_routes_long_name['route_id'].apply(lambda x: color_map[x])

In [55]:
itinerary_routes_long_name['dropdown_route'] = itinerary_routes_long_name.apply(lambda x: str(x['route_id'])+" - "+str(x['route_long_name']), axis=1)

In [56]:
itineraries_reduced_with_long_name = itineraries_reduced.merge(gtfs_routes[['route_id','route_long_name']].astype(str), how='left', on='route_id')

In [57]:
itineraries_reduced_with_long_name.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo,route_long_name
0,0,5974,1254,3982,738,1443.641433,0,WALK,WALK,,Origin,600,NW Bridge Ave & Springville,"LINESTRING (-122.77528 45.58186, -122.77535 45...",16 to 90 to 72,NaN
1,0,5974,1254,3982,738,1443.641433,1,16,BUS,600,NW Bridge Ave & Springville,9888,600 Block NW Naito Parkway,"LINESTRING (-122.77152 45.58528, -122.77149 45...",16 to 90 to 72,Front Ave/St Helens Rd
2,0,5974,1254,3982,738,1443.641433,2,WALK,WALK,9888,600 Block NW Naito Parkway,8339,Old Town/Chinatown MAX Station,"LINESTRING (-122.67258 45.52747, -122.67255 45...",16 to 90 to 72,NaN


In [58]:
itineraries_reduced_with_long_name['route_long_name'] = itineraries_reduced_with_long_name['route_long_name'].fillna(value='WALK')

In [59]:
itineraries_reduced_with_long_name['dropdown_route'] = itineraries_reduced_with_long_name.apply(lambda x: str(x['route_id'])+" - "+str(x['route_long_name']), axis=1)

In [ ]:
itineraries_reduced_with_long_name['route_color'] = itineraries_reduced_with_long_name['route_id'].apply(lambda x: color_map[x])

In [60]:
gdf_points.to_file("gis/export/origin_destination_points.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_points.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_points.geojson",ExtraArgs={'ACL':'public-read'})

In [61]:
itineraries_reduced_with_long_name.to_file("gis/export/itineraries_reduced_with_long_name.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itineraries_reduced_with_long_name.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries_reduced_with_long_name.geojson",ExtraArgs={'ACL':'public-read'})

In [62]:
itinerary_routes_long_name.to_file("gis/export/itinerary_routes_long_name.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itinerary_routes_long_name.geojson", "meysohn-sandbox", "trimet_trip_planner/itinerary_routes_long_name.geojson",ExtraArgs={'ACL':'public-read'})

In [63]:
origin_destination_centriod = LineString(list(gdf_points['points'].to_numpy())).centroid

In [64]:
origin_destination_centriod_gdf = gpd.GeoDataFrame(pd.DataFrame(['origin_dest_centroid'], columns=['id']),crs="EPSG:4326", geometry=[origin_destination_centriod])
origin_destination_centriod_gdf

,id,geometry
0,origin_dest_centroid,POINT (-122.67767 45.53151)


In [65]:
origin_destination_centriod_gdf.to_file("gis/export/origin_destination_centriod.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_centriod.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_centriod.geojson",ExtraArgs={'ACL':'public-read'})